**PYSPARK SQL FUNCTIONS: strings**

In [24]:
import pandas as pd
data = {
  'nombre': ['Juan Pérez', 'María López', 'Carlos Gómez'],
  'puesto': ['Desarrollador', 'Analista de datos', 'Gerente de proyecto'],
  'departamento': ['Técnico', 'Técnico', 'Gestión'],
  'antigüedad': ['1.2345', '3.5', '1.8'],
}
df = pd.DataFrame(data)
df.head()

,nombre,puesto,departamento,antigüedad
0,Juan Pérez,Desarrollador,Técnico,1.2345
1,María López,Analista de datos,Técnico,3.5
2,Carlos Gómez,Gerente de proyecto,Gestión,1.8


In [25]:
df.to_csv('empleados.csv')

In [26]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('appName').getOrCreate()
empleados = spark.read.csv('empleados.csv', header=True, inferSchema=False)
empleados.show()

+---+------------+-------------------+------------+----------+
|_c0|      nombre|             puesto|departamento|antigüedad|
+---+------------+-------------------+------------+----------+
|  0|  Juan Pérez|      Desarrollador|     Técnico|    1.2345|
|  1| María López|  Analista de datos|     Técnico|       3.5|
|  2|Carlos Gómez|Gerente de proyecto|     Gestión|       1.8|
+---+------------+-------------------+------------+----------+



24/05/09 09:46:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre, puesto, departamento, antigüedad
 Schema: _c0, nombre, puesto, departamento, antigüedad
Expected: _c0 but found: 
CSV file: file:///Users/asaucedov/workspaces/python3/librerias-python/pyspark/empleados.csv


In [27]:
empleados.dtypes

[('_c0', 'string'),
 ('nombre', 'string'),
 ('puesto', 'string'),
 ('departamento', 'string'),
 ('antigüedad', 'string')]

In [28]:
# concat, split, instr, left, replace
import pyspark.sql.functions as F
empleados = empleados.withColumn('dpto_puesto', F.concat_ws('-', 'puesto', 'departamento'))
# split string into arrays
empleados = empleados.withColumn('nombres', F.split('nombre', ' '))
# index of substr
empleados = empleados.withColumn('index_of_space', F.instr(F.col('nombre'), ' '))
# extract substr
empleados = empleados.withColumn('nombre_ppal', F.left(F.col('nombre'), F.col('index_of_space')))
# replace text in column
empleados = empleados.withColumn('departamento', F.replace(F.col('departamento'), F.lit('Técnico'), F.lit('IT')))
empleados.show()

+---+------------+-------------------+------------+----------+--------------------+---------------+--------------+-----------+
|_c0|      nombre|             puesto|departamento|antigüedad|         dpto_puesto|        nombres|index_of_space|nombre_ppal|
+---+------------+-------------------+------------+----------+--------------------+---------------+--------------+-----------+
|  0|  Juan Pérez|      Desarrollador|          IT|    1.2345|Desarrollador-Téc...|  [Juan, Pérez]|             5|      Juan |
|  1| María López|  Analista de datos|          IT|       3.5|Analista de datos...| [María, López]|             6|     María |
|  2|Carlos Gómez|Gerente de proyecto|     Gestión|       1.8|Gerente de proyec...|[Carlos, Gómez]|             7|    Carlos |
+---+------------+-------------------+------------+----------+--------------------+---------------+--------------+-----------+



24/05/09 09:46:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre, puesto, departamento, antigüedad
 Schema: _c0, nombre, puesto, departamento, antigüedad
Expected: _c0 but found: 
CSV file: file:///Users/asaucedov/workspaces/python3/librerias-python/pyspark/empleados.csv


In [29]:
# regexp_replace
empleados = empleados.withColumn('solo_letras_y_numeros', F.regexp_replace(F.col('nombre'), r'[^a-z]', '_'))
empleados.show()

+---+------------+-------------------+------------+----------+--------------------+---------------+--------------+-----------+---------------------+
|_c0|      nombre|             puesto|departamento|antigüedad|         dpto_puesto|        nombres|index_of_space|nombre_ppal|solo_letras_y_numeros|
+---+------------+-------------------+------------+----------+--------------------+---------------+--------------+-----------+---------------------+
|  0|  Juan Pérez|      Desarrollador|          IT|    1.2345|Desarrollador-Téc...|  [Juan, Pérez]|             5|      Juan |           _uan___rez|
|  1| María López|  Analista de datos|          IT|       3.5|Analista de datos...| [María, López]|             6|     María |          _ar_a___pez|
|  2|Carlos Gómez|Gerente de proyecto|     Gestión|       1.8|Gerente de proyec...|[Carlos, Gómez]|             7|    Carlos |         _arlos___mez|
+---+------------+-------------------+------------+----------+--------------------+---------------+-------

24/05/09 09:46:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre, puesto, departamento, antigüedad
 Schema: _c0, nombre, puesto, departamento, antigüedad
Expected: _c0 but found: 
CSV file: file:///Users/asaucedov/workspaces/python3/librerias-python/pyspark/empleados.csv


In [30]:
# convert text to number
empleados = empleados.withColumn('num_años', F.to_number(F.col('antigüedad'), F.lit('9999.9999')))
empleados.show()

24/05/09 09:46:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre, puesto, departamento, antigüedad
 Schema: _c0, nombre, puesto, departamento, antigüedad
Expected: _c0 but found: 
CSV file: file:///Users/asaucedov/workspaces/python3/librerias-python/pyspark/empleados.csv


+---+------------+-------------------+------------+----------+--------------------+---------------+--------------+-----------+---------------------+--------+
|_c0|      nombre|             puesto|departamento|antigüedad|         dpto_puesto|        nombres|index_of_space|nombre_ppal|solo_letras_y_numeros|num_años|
+---+------------+-------------------+------------+----------+--------------------+---------------+--------------+-----------+---------------------+--------+
|  0|  Juan Pérez|      Desarrollador|          IT|    1.2345|Desarrollador-Téc...|  [Juan, Pérez]|             5|      Juan |           _uan___rez|  1.2345|
|  1| María López|  Analista de datos|          IT|       3.5|Analista de datos...| [María, López]|             6|     María |          _ar_a___pez|  3.5000|
|  2|Carlos Gómez|Gerente de proyecto|     Gestión|       1.8|Gerente de proyec...|[Carlos, Gómez]|             7|    Carlos |         _arlos___mez|  1.8000|
+---+------------+-------------------+------------+-

In [45]:
# Show sql information
empleados.select(
  (F.current_catalog()),
  (F.current_database()),
  (F.current_date()),
  (F.current_schema()),
  (F.current_timestamp()),
  (F.current_timezone()),
  (F.current_user()),
).show()


+-----------------+------------------+--------------+------------------+--------------------+------------------+--------------+
|current_catalog()|current_database()|current_date()|current_database()| current_timestamp()|current_timezone()|current_user()|
+-----------------+------------------+--------------+------------------+--------------------+------------------+--------------+
|    spark_catalog|           default|    2024-05-09|           default|2024-05-09 09:55:...|     Europe/Madrid|     asaucedov|
|    spark_catalog|           default|    2024-05-09|           default|2024-05-09 09:55:...|     Europe/Madrid|     asaucedov|
|    spark_catalog|           default|    2024-05-09|           default|2024-05-09 09:55:...|     Europe/Madrid|     asaucedov|
+-----------------+------------------+--------------+------------------+--------------------+------------------+--------------+

